In [1]:
!pip install klib sentencepiece transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 28.0 MB/s eta 0:00:00


In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00


In [1]:
import pandas as pd
import torch
from datasets import load_dataset
from transformers import T5ForConditionalGeneration, AutoTokenizer, pipeline,DataCollatorForSeq2Seq,TrainingArguments,Trainer

In [2]:
def gpu_check():
  torch.__version__
  #If ther 's a GPU available...
  if torch.cuda.is_available():
    #Tell Pytorch to use GPU.
    device = torch.device("cuda")
    print("There are %d GPU(s) avalilable." % torch.cuda.device_count())
    print('We will use the GPU:',torch.cuda.get_device_name(0))
    #If not..
  else:
    print('No GPU available,using the CPU instead.')
    device = torch.device("cpu")
    return device

In [3]:
device=gpu_check()
print(device)

There are 1 GPU(s) avalilable.
We will use the GPU: Tesla T4
None


In [4]:
model_name = 'erfan226/persian-t5-paraphraser'
tokenizer = AutoTokenizer.from_pretrained(model_name,model_max_length=100, add_special_tokens = True)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

In [9]:
!unzip '/content/Persian Paraphrase Dataset.zip'

Archive:  /content/Persian Paraphrase Dataset.zip
replace Chunk_4_official.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [13]:
df1=pd.read_csv('/content/Chunk_4_official.csv')
df2=pd.read_csv('/content/Chunk_4_official.csv')
df1.columns
df2.columns

Index(['prefix', 'input_text', 'target_text'], dtype='object')

In [14]:
df=pd.concat([df1[['input_text','target_text']],df2[['input_text','target_text']]],axis=0)
df.head()

,input_text,target_text
0,بنابراین من به شما نیاز دارم که از زندگی شخصی ...,من به شما نیاز دارم که از زندگی شخصی من دور بم...
1,احتمالا شما مادر بسیار سلطه پذیری داشتید.,احتمالاً به این دلیل که شما یک مادر بسیار مسلط...
2,وحشتناک به نظر می رسد.,وحشتناک به نظر می رسد
3,نه، نه، این نیست.,نه، اون نه.
4,می دانید چه کسی پوست بدی داشت و چه کسی موهای ب...,می دانید، پوست بد و موی بد.


In [15]:
from sklearn.model_selection import train_test_split

In [17]:
train_df, test_df = train_test_split(df[:1000], test_size=0.2, random_state=101)

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

train_df.to_csv("/content/train.csv", sep="\t", encoding="utf-8", index=False)
test_df.to_csv("/content/test.csv", sep="\t", encoding="utf-8", index=False)

In [18]:
data_files = {
    "train": "/content/train.csv",
    "test": "/content/test.csv",
}

dataset = load_dataset("csv", data_files=data_files, delimiter="\t")
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [19]:
def preprocess_function(examples):
    padding = "max_length"
    max_length = 100

    inputs = [ex for ex in examples["input_text"]]
    targets = [ex for ex in examples["target_text"]]
    model_inputs = tokenizer(inputs, max_length=max_length, padding=padding, truncation=True)
    labels = tokenizer(targets, max_length=max_length, padding=padding, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [20]:
tokenized_data = dataset.map(preprocess_function, batched=True, desc="Running tokenizer")

Running tokenizer:   0%|          | 0/800 [00:00<?, ? examples/s]

Running tokenizer:   0%|          | 0/200 [00:00<?, ? examples/s]

In [21]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['input_text', 'target_text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 800
    })
    test: Dataset({
        features: ['input_text', 'target_text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 200
    })
})

In [22]:
!pip install nltk
!pip install rouge

In [24]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=tokenizer.pad_token_id,
    return_tensors="pt")

In [25]:
training_args = TrainingArguments(
    output_dir="/output/",
    learning_rate=2e-5,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy ="epoch",
    save_strategy ="epoch",
    load_best_model_at_end=True,
    push_to_hub=False
)

In [28]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [29]:
print("شروع آموزش...")
trainer.train()

شروع آموزش...


Epoch,Training Loss,Validation Loss
1,No log,0.281801
2,No log,0.256220
3,No log,0.264415
4,0.417600,0.243066
5,0.417600,0.237335
6,0.417600,0.233662
7,0.346000,0.232742
8,0.346000,0.232018
9,0.346000,0.231225
10,0.302200,0.231271


TrainOutput(global_step=1600, training_loss=0.3512918567657471, metrics={'train_runtime': 816.7004, 'train_samples_per_second': 9.796, 'train_steps_per_second': 1.959, 'total_flos': 1069841203200000.0, 'train_loss': 0.3512918567657471, 'epoch': 10.0})

In [30]:
trainer.evaluate()

{'eval_loss': 0.23122543096542358,
 'eval_runtime': 3.7654,
 'eval_samples_per_second': 53.115,
 'eval_steps_per_second': 10.623,
 'epoch': 10.0}

In [31]:
trainer.save_model('final_Model')

In [32]:
pipe = pipeline(task='text2text-generation', model='./final_Model', tokenizer=tokenizer)

In [62]:
# def paraphrase(text):
#   for j in range(2):
#     out = pipe(text, do_sample=True, max_length=100)[0]['generated_text']
#     print(f"Paraphrase {j} :", out)

text = "معاون توسعه مدیریت و منابع استاندار تهران گفت: ساعت آغاز به کار دستگاه‌های اجرایی ملی مستقر در شهر تهران از ۱۸ شهریور به صورت شناور خواهد بود."
print("Original:", text)
print("paraphrase:",pipe(text, do_sample=True,max_length=100)[0]['generated_text'])
# paraphrase(text)

Original: معاون توسعه مدیریت و منابع استاندار تهران گفت: ساعت آغاز به کار دستگاه‌های اجرایی ملی مستقر در شهر تهران از ۱۸ شهریور به صورت شناور خواهد بود.
paraphrase: ساعت آغاز به کار دستگاه های اجرایی ملی مستقر در شهر تهران به صورت شناور است.


In [68]:
!zip -r persian_paraphrasing.zip 'final_Model'

updating: final_Model/ (stored 0%)
  adding: final_Model/generation_config.json (deflated 29%)
  adding: final_Model/training_args.bin (deflated 49%)
  adding: final_Model/special_tokens_map.json (deflated 86%)
  adding: final_Model/config.json (deflated 48%)
  adding: final_Model/tokenizer.json (deflated 83%)
  adding: final_Model/pytorch_model.bin (deflated 7%)
  adding: final_Model/tokenizer_config.json (deflated 81%)
